<a href="https://colab.research.google.com/github/cserock/colab-examples/blob/main/08_LangGraph_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup
For Colab, the helper folders need to be copied over from the repo. The below cell does this automatically.

In [ ]:
%%bash

# Check if the environment variable exists
if [ -n "$COLAB_RELEASE_TAG" ] || [ -n "$COLAB_GPU" ]; then
    echo "Running on Google Colab. Cloning repository into temp folder..."
    git clone https://github.com/TuebingenAICenter/agent-tutorial.git /tmp/tmp_repo
    echo "Moving all helpers to project root..."
    mv /tmp/tmp_repo/chat_with_X_utils .
    mv /tmp/tmp_repo/images .
    mv /tmp/tmp_repo/env.example ./.env
    mv /tmp/tmp_repo/requirements.txt .
else
    echo "Not running on Google Colab. Skipping git clone."
fi

# The installation block runs regardless of environment.
echo "Checking for requirements.txt and installing required packages..."

# Check if requirements.txt exists in the current directory
if [ -f "requirements.txt" ]; then
    # Attempt to install with 'uv', and if it fails (exit code != 0), use 'pip' as a fallback.
    if command -v uv &> /dev/null; then
        echo "uv detected. Installing with uv..."
        uv pip install -r requirements.txt
    else
        echo "Installing with pip..."
        pip install -r requirements.txt
    fi
else
    echo "ERROR! requirements.txt not found! Please check for errors..."
fi

## 환경변수 파일(.env) 설정하기
/content/drive/MyDrive/lg-dx/에 아래 내용으로 .env 파일을 작성한 후 업로드합니다.  

OPENAI_API_KEY=sk-xxxx  
LANGCHAIN_TRACING_V2="true"  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
LANGCHAIN_API_KEY="lsv2_xxxx  
LANGCHAIN_PROJECT="lg-dx"


Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

python-dotenv 라이브러리 설치

In [ ]:
!pip install python-dotenv

환경변수 파일 로드 및 확인

In [ ]:
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/lg-dx/.env', override=True)

import os
openai_api_key = os.environ.get('OPENAI_API_KEY')
print("openai_api_key : " + openai_api_key)
langchain_api_key = os.environ.get('LANGCHAIN_API_KEY')
print("langchain_api_key : " + langchain_api_key)

# Example 01: Basic LangGraph Chatbot & Tool

**What:** A simple code example for a basic chatbot that has access to a multiply function.

**Why:** To show you (i) basic LangGraph flow, (ii) how all the concepts we discussed stitch together

**Live:** Follow along with notebook 01 from our repository. Ideally, we can all run this easily $\implies$ setup for advanced examples later.

## What we plan to create:

<img src="https://github.com/TuebingenAICenter/agent-tutorial/blob/main/images/nb-1-overv.png?raw=1">

## What it looks like in LangGraph

<img src="https://github.com/TuebingenAICenter/agent-tutorial/blob/main/images/cond-nb-1.png?raw=1">

## Setup

### Importing necessary packages

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

from langchain_core.tools import tool

<!-- ### Setting API key and initializing LLM -->
### Initializing LLM

In [ ]:
print(os.getenv("OPENAI_API_KEY"))

In [ ]:
llm = ChatOpenAI(
  model="gpt-4.1-mini",
  temperature=0.7,
  openai_api_key=os.getenv("OPENAI_API_KEY"),
)

### Helper functions

In [ ]:
from chat_with_X_utils.print_utils import (
    print_messages_from_stream_event as _print_messages_from_stream_event,
    print_messages_from_state as _print_messages_from_state,
)

## Basic chatbot


#### [Reducers](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers)
Reducers define how keys of the state should be updated (instead of overridden).

In [ ]:
from langgraph.graph.message import add_messages

### [State](https://langchain-ai.github.io/langgraph/concepts/low_level/#state)

A typed dictionary that all nodes (and conditional edges) operate on. It stores information that persists between nodes.


In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

### [StateGraph](https://langchain-ai.github.io/langgraph/concepts/low_level/#stategraph)

The "design" graph that operates on the `State` (not compiled yet!).

Uses a `State` object to define the structure of our agentic system (which is a state machine).

In [ ]:
design_graph = StateGraph(State)

### Tools

Tools are functions designed to be called by an LLM.

- We can easily build our own using the [`@tool()` decorator](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.convert.tool.html). which wraps a function to make it easily callable.

- What's nice is that we can provide the docs_string as a "readable" to the LLM along with the function name using `@tool(parse_docstring=True)`.

Note that the LLM needs to support tool calling for this functionality.

#### Tool API

In [ ]:
@tool(parse_docstring=True) #parse_docstring=True allows to add descriptions for the arguments
def multiply_two_integers(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a: The first integer to multiply.
        b: The second integer to multiply.
    """
    return a*b

multiply_two_integers.args_schema.model_json_schema()

In [ ]:
tools = [multiply_two_integers]
llm_with_tools = llm.bind_tools(tools)

### [Nodes](https://langchain-ai.github.io/langgraph/concepts/low_level/#nodes)

Functions that operate on `State` and ouptut updates to it. Usually LLM calls, that may use tools.

Here we [invoke](https://python.langchain.com/docs/how_to/lcel_cheatsheet/#invoke-a-runnable) the LLM, which is a [runnable](https://python.langchain.com/docs/concepts/runnables/). A runnable is the foundational high-level LangChain abstraction that represents any language model, output parser, retriever, or compiled LangGraph graphs, amongst others. Invoking it means accepting an input and returning an output ("run button").

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

design_graph.add_node("chatbot", chatbot)

#### [ToolNode](https://langchain-ai.github.io/langgraph/concepts/low_level/#edges)

A pre-built tool box which becomes a node in our graph. Runs the tools called in the last [AIMessage](https://python.langchain.com/docs/concepts/messages/#aimessage) and appends the resulting [ToolMessage(s)](https://python.langchain.com/docs/concepts/messages/#toolmessage) to the "messages" state key (or a custom key passed into ToolNode)

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools=[multiply_two_integers])
design_graph.add_node("multiply_tool", tool_node)


### [Edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#end-node)

Specifies how an agentic system should transition between nodes.

#### [Conditional edges](https://langchain-ai.github.io/langgraph/concepts/low_level/#conditional-edges)

Defined by a function operating on the state which outputs the next node to transition to.
Here, [`tools_condition`](https://langchain-ai.github.io/langgraph/reference/agents/?h=toolnode#langgraph.prebuilt.tool_node.tools_condition) outputs either "tools" if a tool call is detected in the last [AIMessage](https://python.langchain.com/docs/concepts/messages/#aimessage) or `END` if it isn't.

In [ ]:
from langgraph.prebuilt import tools_condition

# tools_condition checks the state for a tool call,
# if a tool call exists, tools_condition == "tools"
# else, tools_condition == "__end__"

def multiply_if_llm_wants_to(state: State):
    nxt = tools_condition(state)
    if nxt == END:
        return END
    return "multiply_tool"


# graph_builder.add_conditional_edges("chatbot", tools_condition)
design_graph.add_conditional_edges("chatbot", multiply_if_llm_wants_to, ["multiply_tool", END])

design_graph.add_edge("multiply_tool", "chatbot")
design_graph.add_edge(START, "chatbot")

### [Checkpointer](https://langchain-ai.github.io/langgraph/concepts/memory/#short-term-memory)

This is an in-memory checkpointer (just uses RAM) so that the LLM does not forget what we said in the previous messages. We can also use an SQL DB for this or whatever we like.

Note that we define a `config` object here, which contains a `thread_id`. This governs the thread for which we checkpoint.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
config = {"configurable": {"thread_id": "1"}}
memory = MemorySaver()

### Compile and Test

Now `graph` is an instance of `CompiledStateGraph`, and we can run things.

In [ ]:
compiled_graph = design_graph.compile(checkpointer=memory)

try:
    from IPython.display import Image, display
    # This is remote call and may fail due to rate limits
    display(Image(compiled_graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Mermaid rendering failed, trying ascii art")
    print(compiled_graph.get_graph().print_ascii())

In [ ]:
def stream_graph_updates(user_input: str, _printed: set):
    for event in compiled_graph.stream({"messages": [{"role": "user", "content": user_input}]}, config, stream_mode="values"):
        _print_messages_from_stream_event(event, _printed)


_printed = set()
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")s
        break

    stream_graph_updates(user_input, _printed)